### Chat with your LinkedIn Profile


In [14]:
# From the agents library import necessary components
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession

#### Load environment

In [8]:
load_dotenv(override=True)

True

In [9]:
# Define model
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
import os

# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using the local LLM server client
# thinking mode: gpt-oss. takes time
model = OpenAIChatCompletionsModel(model = "llama3.2:latest",openai_client= client)

#### Read Linkedin PDF to text

Parsing linkedin PDF to text and creating system prompt

In [11]:
from pypdf import PdfReader

reader = PdfReader("me.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name="Priyabrata"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt

"You are acting as Priyabrata. You are answering questions on Priyabrata's website, particularly questions related to Priyabrata's career, background, skills and experience. Your responsibility is to represent Priyabrata for interactions on the website as faithfully as possible. You are given a summary of Priyabrata's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Priyabrata. I'm an entrepreneur, bioinformatician and data scientist. I'm originally from india.\nI love all foods, particularly Indian food, biryani, but strangely I'm repelled by almost all forms of cheese. \nI'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n9938908025 (Mobile)\npb.panigrah

In [12]:
# Define Agent
# Create an agent using the local model
local_agent = Agent(name="LinkedInAgent",
                    instructions=system_prompt,
                    model=model)

### Gradio chat app with local llm

In [15]:
session = SQLiteSession("linkedin_agent_session.db")

async def chat(message, history):
    result = await Runner.run(local_agent, message, session=session)
    return result.final_output

import gradio as gr
gr.ChatInterface(
    chat,
    title="Priyabrata LinkedIn Agent",
    description="Chat with Priyabrata's LinkedIn agent."
).launch()


/mnt/c/Users/PriyabrataPanigrahi/Downloads/ai/aienv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
